<a href="https://colab.research.google.com/github/Rmm6699/HW/blob/master/W7_TA_RecSys_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 永豐銀行 AI人才培育課程--個人化商品推薦系統 #2 <a class="anchor" id="0"></a>

## 目錄 Table of Contents <a class="anchor" id="0.1"></a>

0.	[Loading & Preprocessing Data](#0)
1.	[Factorization Machine, FM](#1)   
2.	[FM-supported Neural Networks, FNN](#2)
3.	[Product-based Neural Networks](#3)
    - 3.1	[Inner Product Neural Networks, IPNN](#3.1)
    - 3.2	[Outer Product Neural Networks, OPNN](#3.2)


4.	[Wide & Deep](#4)
5.	[Deep & Cross Network](#5)
6.	[Neural Factorization Machines, NFM](#6)
7. [DeepFM](#7)

In [ ]:
!pip install deepctr_torch

# 0. Loading & Preprocessing Data <a id="0"></a> <br>

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import log_loss, roc_auc_score

In [ ]:
# Load data

data = pd.read_csv('./data/criteo_sample.txt')

sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I' + str(i) for i in range(1, 14)]

data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )
target = ['label']

In [ ]:
data

# I1~I13 為 數值型資料
# C1~C26 為 類別型資料

# label為該廣告是否被點擊 (1 if clicked, 0 if non-clicked)
# user_id 和 item_id 也可以視為 Sparse Feature

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,0.0,3,260.0,0.0,17668.0,0.0,0.0,33.0,0.0,...,e5ba7672,87c6f83c,-1,-1,0429f84b,-1,3a171ecb,c0d61a5c,-1,-1
1,0,0.0,-1,19.0,35.0,30251.0,247.0,1.0,35.0,160.0,...,d4bb7bd8,6fc84bfb,-1,-1,5155d8a3,-1,be7c41b4,ded4aac9,-1,-1
2,0,0.0,0,2.0,12.0,2013.0,164.0,6.0,35.0,523.0,...,e5ba7672,675c9258,-1,-1,2e01979f,-1,bcdee96c,6d5d1302,-1,-1
3,0,0.0,13,1.0,4.0,16836.0,200.0,5.0,4.0,29.0,...,e5ba7672,52e44668,-1,-1,e587c466,-1,32c7478e,3b183c5c,-1,-1
4,0,0.0,0,104.0,27.0,1990.0,142.0,4.0,32.0,37.0,...,e5ba7672,25c88e42,21ddcdc9,b1252a9d,0e8585d2,-1,32c7478e,0d4a6d1a,001f3601,92c878de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0.0,0,113.0,3.0,3036.0,575.0,2.0,3.0,214.0,...,07c540c4,9880032b,21ddcdc9,5840adea,34cc61bb,c9d4222a,32c7478e,e5ed7da2,ea9a246c,984e0db0
196,1,0.0,1,1.0,1.0,1607.0,12.0,1.0,12.0,15.0,...,1e88c74f,3972b4ed,-1,-1,d1aa4512,-1,32c7478e,9257f75f,-1,-1
197,1,1.0,0,6.0,3.0,0.0,0.0,19.0,3.0,3.0,...,3486227d,5aed7436,54591762,a458ea53,4a2c3526,-1,32c7478e,1793a828,e8b83407,1a02cbe1
198,0,0.0,22,6.0,22.0,203.0,153.0,80.0,18.0,508.0,...,3486227d,13145934,55dd3565,5840adea,bf647035,-1,32c7478e,1481ceb4,e8b83407,988b0775


In [ ]:
# 1.Label Encoding for sparse features,and do simple Transformation for dense features

# For Sparse Features
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
    
# For Dense Features
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])
data

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,0.000000,0.001332,0.092362,0.000000,0.034825,0.000000,0.000000,0.673469,0.000000,...,8,66,0,0,3,0,1,96,0,0
1,0,0.000000,0.000000,0.006750,0.402299,0.059628,0.117284,0.003322,0.714286,0.154739,...,7,52,0,0,47,0,7,112,0,0
2,0,0.000000,0.000333,0.000710,0.137931,0.003968,0.077873,0.019934,0.714286,0.505803,...,8,49,0,0,25,0,6,53,0,0
3,0,0.000000,0.004664,0.000355,0.045977,0.033185,0.094967,0.016611,0.081633,0.028046,...,8,37,0,0,156,0,0,32,0,0
4,0,0.000000,0.000333,0.036945,0.310345,0.003922,0.067426,0.013289,0.653061,0.035783,...,8,14,5,3,9,0,0,5,1,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0.000000,0.000333,0.040142,0.034483,0.005984,0.273029,0.006645,0.061224,0.206963,...,0,74,5,1,30,5,0,118,17,48
196,1,0.000000,0.000666,0.000355,0.011494,0.003168,0.005698,0.003322,0.244898,0.014507,...,1,25,0,0,138,0,0,68,0,0
197,1,0.027027,0.000333,0.002131,0.034483,0.000000,0.000000,0.063123,0.061224,0.002901,...,4,40,17,2,41,0,0,12,16,11
198,0,0.000000,0.007662,0.002131,0.252874,0.000400,0.072650,0.265781,0.367347,0.491296,...,4,7,18,1,123,0,0,10,16,49


In [ ]:
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
# 2. count #unique features for each sparse field,and record dense feature field name

embedding_dim = 4
# SparseFeat(類別型特徵名稱, 獨有的特徵數, 轉換後的特徵表示向量維度)
sparse = [SparseFeat(feat, data[feat].nunique(), embedding_dim=embedding_dim) for feat in sparse_features]

# DenseFeat(類別型特徵名稱)
dense = [DenseFeat(feat) for feat in dense_features]

fixlen_feature_columns = sparse + dense

In [ ]:
fixlen_feature_columns

[SparseFeat(name='C1', vocabulary_size=27, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C1', group_name='default_group'),
 SparseFeat(name='C2', vocabulary_size=92, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C2', group_name='default_group'),
 SparseFeat(name='C3', vocabulary_size=172, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C3', group_name='default_group'),
 SparseFeat(name='C4', vocabulary_size=157, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C4', group_name='default_group'),
 SparseFeat(name='C5', vocabulary_size=12, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C5', group_name='default_group'),
 SparseFeat(name='C6', vocabulary_size=7, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C6', group_name='default_group'),
 SparseFeat(name='C7', vocabulary_size=183, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C7', group_name='default_group'),
 SparseFeat

### feature_columns
linear_feature_columns: An iterable containing all the features used by linear part of the model.

dnn_feature_columns: An iterable containing all the features used by deep part of the model.

|             | Linear_feature_columns | Dnn_feature_columns |   |   |
|-------------|------------------------|---------------------|---|---|
| FM          |            V           |          V          |   |   |
| FNN         |                        |          V          |   |   |
| IPNN        |                        |          V          |   |   |
| OPNN        |                        |          V          |   |   |
| Wide & Deep |            V           |          V          |   |   |
| Deep Cross  |            V           |          V          |   |   |
| NFM         |            V           |          V          |   |   |
| DeepFM      |            V           |          V          |   |   |

In [ ]:
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [ ]:
feature_names

['C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21',
 'C22',
 'C23',
 'C24',
 'C25',
 'C26',
 'I1',
 'I2',
 'I3',
 'I4',
 'I5',
 'I6',
 'I7',
 'I8',
 'I9',
 'I10',
 'I11',
 'I12',
 'I13']

In [ ]:
from sklearn.model_selection import train_test_split

# 3.Split training and testing data for model

train, test = train_test_split(data, test_size=0.2, random_state=2022)
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}

In [ ]:
train_model_input

{'C1': 74      0
 132     0
 168     0
 117     0
 137     0
        ..
 183     0
 177    11
 112     0
 173    11
 125    11
 Name: C1, Length: 160, dtype: int64,
 'C2': 74     87
 132    30
 168    18
 117    19
 137    91
        ..
 183    18
 177    74
 112    72
 173    44
 125    10
 Name: C2, Length: 160, dtype: int64,
 'C3': 74      87
 132     74
 168    161
 117     12
 137     64
       ... 
 183    116
 177      3
 112    127
 173    115
 125     41
 Name: C3, Length: 160, dtype: int64,
 'C4': 74     115
 132     64
 168     91
 117     99
 137    125
       ... 
 183      8
 177    148
 112    152
 173      6
 125    134
 Name: C4, Length: 160, dtype: int64,
 'C5': 74     1
 132    6
 168    1
 117    1
 137    4
       ..
 183    1
 177    6
 112    1
 173    1
 125    1
 Name: C5, Length: 160, dtype: int64,
 'C6': 74     3
 132    4
 168    5
 117    2
 137    4
       ..
 183    5
 177    3
 112    4
 173    4
 125    4
 Name: C6, Length: 160, dtype: int64,
 'C7': 74 

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# 1. Factorization Machine, FM <a id="1"></a> <br>

$$
\hat{y}(x) = w_0 + \sum_{i=1}^{n}w_i x_i + \sum_{i=1}^{n} \sum_{j=i+1}^{n} \langle \textbf v_i, \textbf v_j \rangle x_i x_j
$$

$$
\sum_{i=1}^{n} \sum_{j=i+1}^{n} \langle \textbf v_i, \textbf v_j \rangle x_i x_j = 
\frac{1}{2} \sum_{f=1}^{k} \Big( \big(\sum_{i=1}^{n} v_f^{(i)} x_i \big)^2 - \sum_{i=1}^{n}v_f^{(i) 2} x_i^2 \Big) = 
\frac{1}{2} \sum_{f=1}^{} \Big( S_{1,f}^2 - S_{2,f} \Big) =
\frac{1}{2} \Big( S_{1}^2 - S_{2} \Big)
$$
n = numbers of features, k = embedding_dim

In [ ]:
from deepctr_torch.models.basemodel import BaseModel

class FM(BaseModel):
    """Factorization Machine models pairwise (order-2) feature interactions
     without linear term and bias.
      Input shape
        - 3D tensor with shape: ``(batch_size,field_size,embedding_size)``.
      Output shape
        - 2D tensor with shape: ``(batch_size, 1)``.
      References
        - [Factorization Machines](https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf)
    """

    def __init__(self,
                 linear_feature_columns, dnn_feature_columns, embedding_dim=4,
                 l2_reg_linear=0.00001, l2_reg_embedding=0.00001, init_std=0.0001, seed=1024, 
                 task='binary', device='cpu', gpus=None):
        super(FM, self).__init__(linear_feature_columns, dnn_feature_columns, l2_reg_linear=l2_reg_linear,
                                 l2_reg_embedding=l2_reg_embedding, init_std=init_std, seed=seed, task=task,
                                 device=device, gpus=gpus)
        
        # If input contains dense features
        if len(self.linear_model.dense_feature_columns) > 0:
            self.V = torch.nn.Parameter(torch.Tensor(len(self.linear_model.dense_feature_columns), embedding_dim))
            torch.nn.init.normal_(self.V, mean=0, std=init_std)
            
        self.embedding_dim = embedding_dim
        self.to(device)
        
        
    def forward_cross(self, inputs):
        
        fm_input = inputs
        square_of_sum = torch.pow(torch.sum(fm_input, dim=1, keepdim=True), 2)
        sum_of_square = torch.sum(fm_input * fm_input, dim=1, keepdim=True)
        cross_term = square_of_sum - sum_of_square
        cross_term = 0.5 * torch.sum(cross_term, dim=2, keepdim=False)

        return cross_term

    def forward(self, X):
        
        # Linear term
        linear = self.linear_model(X) # shape=(batch_size, 1)
        
        # Cross_term
        sparse_embedding_list, dense_value_list = self.input_from_feature_columns(X, self.dnn_feature_columns, self.embedding_dict)
        
        # sparse 已經過 embedding_layer , 已乘上權重
        # dense 尚未乘上權重
        
        # Case 1 Features包含Dense與Sparse features
        if len(dense_value_list) > 0 and len(sparse_embedding_list) > 0:
            dense_embedding = torch.cat(dense_value_list, dim=-1) # shape=(batch_size, num_dense=13)
            dense_embedding = dense_embedding.unsqueeze(-1) # shape=(batch_size, num_dense=13, 1)

            # self.V.shape = (num_dense=13, k=4)
            dense_embedding = dense_embedding * self.V # 乘上權重 (batch_size, num_dense=13, k=4)
            
            cross_input = torch.cat(sparse_embedding_list+[dense_embedding], dim=1) # shape=(batch_size, n=26+13=39, k=4) 
            # cross_input = XV
            cross = self.forward_cross(cross_input) # shape=(batch_size, 1)

        # Case 2 Features只有Dense features
        elif len(dense_value_list) > 0:
            cross_input = (torch.cat(dense_value_list, dim=-1).unsqueeze(-1) * self.V).reshape(logit.shape[0], -1) # shape=(batch_size, n=13, k=4)
            cross = self.forward_cross(cross_input) # shape=(batch_size, 1)
            
        # Case 3 Features只有Sparse features
        elif len(sparse_embedding_list) > 0:
            cross_input = torch.cat(sparse_embedding_list, dim=1) # shape=(batch_size, n=26, k=4)
            cross = self.forward_cross(cross_input) # shape=(batch_size, 1)

        logit = linear + cross
        y_pred = self.out(logit) # sigmoid(logit) if task='binary' else logit + bias (task=ratings)

        return y_pred

**sparse_embedding_list** =

                    1. embedding shape=[32, 1, 4],

                    2. embedding shape=[32, 1, 4],
                         
                    .... 
                         
                    26.embedding shape=[32, 1, 4]
                    
*shape = (batch_size, 1, embedding_dim)*



**dense_value_list** =

                    1. vector shape=[32, 1],

                    2. vector shape=[32, 1],
                         
                    .... 
                         
                    13.vector shape=[32, 1]
                    
*shape = (batch_size, 1)*

## Loss and Metrics

loss = [ **binary_crossentropy** (binary), **mse** (regression), **mae** (regression) ]

metrics = [**binary_crossentropy**, **logloss**, **auc**, **mse**, **accuracy**, **acc**]


In [ ]:
model = FM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns, 
            embedding_dim=embedding_dim, task='binary', l2_reg_embedding=1e-5, device=device)

model.compile("adam", loss="binary_crossentropy", metrics=["binary_crossentropy"], ) # "auc"

history = model.fit(train_model_input, train[target].values, batch_size=32, epochs=10, verbose=2, # 0: no message 1: progress bar 2: message
                    validation_split=0.2)

pred_ans = model.predict(test_model_input, 256)

print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

cuda
Train on 128 samples, validate on 32 samples, 4 steps per epoch
Epoch 1/10
0s - loss:  0.6892 - binary_crossentropy:  0.6892 - val_binary_crossentropy:  0.6830
Epoch 2/10
0s - loss:  0.6648 - binary_crossentropy:  0.6648 - val_binary_crossentropy:  0.6727
Epoch 3/10
0s - loss:  0.6427 - binary_crossentropy:  0.6427 - val_binary_crossentropy:  0.6627
Epoch 4/10
0s - loss:  0.6196 - binary_crossentropy:  0.6196 - val_binary_crossentropy:  0.6535
Epoch 5/10
0s - loss:  0.5970 - binary_crossentropy:  0.5970 - val_binary_crossentropy:  0.6454
Epoch 6/10
0s - loss:  0.5743 - binary_crossentropy:  0.5743 - val_binary_crossentropy:  0.6383
Epoch 7/10
0s - loss:  0.5511 - binary_crossentropy:  0.5511 - val_binary_crossentropy:  0.6321
Epoch 8/10
0s - loss:  0.5272 - binary_crossentropy:  0.5272 - val_binary_crossentropy:  0.6268
Epoch 9/10
0s - loss:  0.5022 - binary_crossentropy:  0.5022 - val_binary_crossentropy:  0.6223
Epoch 10/10
0s - loss:  0.4771 - binary_crossentropy:  0.4771 - val

# 2. FM-supported Neural Networks, FNN <a id="2"></a> <br>

In [ ]:
from models import FNN

model = FNN(dnn_feature_columns=dnn_feature_columns, dnn_hidden_units=(128, 128), dnn_activation='relu', 
            task='binary', l2_reg_embedding=1e-5, l2_reg_dnn=0, device=device)

model.compile("adam", loss="binary_crossentropy", metrics=["binary_crossentropy"], )

history = model.fit(train_model_input, train[target].values, batch_size=32, epochs=10, verbose=2,
                    validation_split=0.2)
pred_ans = model.predict(test_model_input, 256)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

cuda
Train on 128 samples, validate on 32 samples, 4 steps per epoch
Epoch 1/10
0s - loss:  0.6848 - binary_crossentropy:  0.6848 - val_binary_crossentropy:  0.6774
Epoch 2/10
0s - loss:  0.6695 - binary_crossentropy:  0.6695 - val_binary_crossentropy:  0.6661
Epoch 3/10
0s - loss:  0.6576 - binary_crossentropy:  0.6576 - val_binary_crossentropy:  0.6538
Epoch 4/10
0s - loss:  0.6423 - binary_crossentropy:  0.6423 - val_binary_crossentropy:  0.6410
Epoch 5/10
0s - loss:  0.6237 - binary_crossentropy:  0.6237 - val_binary_crossentropy:  0.6272
Epoch 6/10
0s - loss:  0.6028 - binary_crossentropy:  0.6028 - val_binary_crossentropy:  0.6122
Epoch 7/10
0s - loss:  0.5779 - binary_crossentropy:  0.5779 - val_binary_crossentropy:  0.5988
Epoch 8/10
0s - loss:  0.5459 - binary_crossentropy:  0.5459 - val_binary_crossentropy:  0.5908
Epoch 9/10
0s - loss:  0.5208 - binary_crossentropy:  0.5208 - val_binary_crossentropy:  0.5921
Epoch 10/10
0s - loss:  0.4899 - binary_crossentropy:  0.4899 - val

# 3. Product-based Neural Networks <a id="3"></a> <br>

## 3.1 Inner Product Neural Networks, IPNN <a id="3.1"></a> <br>

In [ ]:
from deepctr_torch.models import PNN

model = PNN(dnn_feature_columns=dnn_feature_columns, dnn_hidden_units=(128, 128), dnn_activation='relu', 
            use_inner=True, use_outter=False, task='binary', l2_reg_embedding=1e-5, l2_reg_dnn=0, device=device)

model.compile("adam", loss="binary_crossentropy", metrics=["binary_crossentropy"], )

history = model.fit(train_model_input, train[target].values, batch_size=32, epochs=10, verbose=2,
                    validation_split=0.2)
pred_ans = model.predict(test_model_input, 256)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

cuda
Train on 128 samples, validate on 32 samples, 4 steps per epoch
Epoch 1/10
0s - loss:  0.6816 - binary_crossentropy:  0.6816 - val_binary_crossentropy:  0.6773
Epoch 2/10
0s - loss:  0.6717 - binary_crossentropy:  0.6717 - val_binary_crossentropy:  0.6673
Epoch 3/10
0s - loss:  0.6596 - binary_crossentropy:  0.6596 - val_binary_crossentropy:  0.6562
Epoch 4/10
0s - loss:  0.6448 - binary_crossentropy:  0.6448 - val_binary_crossentropy:  0.6437
Epoch 5/10
0s - loss:  0.6258 - binary_crossentropy:  0.6258 - val_binary_crossentropy:  0.6291
Epoch 6/10
0s - loss:  0.6007 - binary_crossentropy:  0.6007 - val_binary_crossentropy:  0.6131
Epoch 7/10
0s - loss:  0.5721 - binary_crossentropy:  0.5721 - val_binary_crossentropy:  0.5984
Epoch 8/10
0s - loss:  0.5386 - binary_crossentropy:  0.5386 - val_binary_crossentropy:  0.5905
Epoch 9/10
0s - loss:  0.5038 - binary_crossentropy:  0.5038 - val_binary_crossentropy:  0.5949
Epoch 10/10
0s - loss:  0.4712 - binary_crossentropy:  0.4712 - val

## 3.2 Outer Product Neural Networks, OPNN <a id="3.2"></a> <br>

In [ ]:
from deepctr_torch.models import PNN

model = PNN(dnn_feature_columns=dnn_feature_columns, dnn_hidden_units=(128, 128), dnn_activation='relu', 
            use_inner=False, use_outter=True, task='binary', l2_reg_embedding=1e-5, l2_reg_dnn=0, device=device)

model.compile("adam", loss="binary_crossentropy", metrics=["binary_crossentropy"], )

history = model.fit(train_model_input, train[target].values, batch_size=32, epochs=10, verbose=2,
                    validation_split=0.2)
pred_ans = model.predict(test_model_input, 256)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

cuda
Train on 128 samples, validate on 32 samples, 4 steps per epoch
Epoch 1/10
0s - loss:  0.6886 - binary_crossentropy:  0.6886 - val_binary_crossentropy:  0.6831
Epoch 2/10
0s - loss:  0.6775 - binary_crossentropy:  0.6775 - val_binary_crossentropy:  0.6726
Epoch 3/10
0s - loss:  0.6668 - binary_crossentropy:  0.6668 - val_binary_crossentropy:  0.6608
Epoch 4/10
0s - loss:  0.6494 - binary_crossentropy:  0.6494 - val_binary_crossentropy:  0.6484
Epoch 5/10
0s - loss:  0.6327 - binary_crossentropy:  0.6327 - val_binary_crossentropy:  0.6336
Epoch 6/10
0s - loss:  0.6082 - binary_crossentropy:  0.6082 - val_binary_crossentropy:  0.6165
Epoch 7/10
0s - loss:  0.5755 - binary_crossentropy:  0.5755 - val_binary_crossentropy:  0.6006
Epoch 8/10
0s - loss:  0.5402 - binary_crossentropy:  0.5402 - val_binary_crossentropy:  0.5904
Epoch 9/10
0s - loss:  0.5065 - binary_crossentropy:  0.5065 - val_binary_crossentropy:  0.5924
Epoch 10/10
0s - loss:  0.4730 - binary_crossentropy:  0.4730 - val

# 4. Wide & Deep <a id="4"></a> <br>

In [ ]:
from deepctr_torch.models import WDL

model = WDL(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 128), dnn_activation='relu', 
            task='binary', l2_reg_embedding=1e-5, l2_reg_dnn=0, device=device)

model.compile("adam", loss="binary_crossentropy", metrics=["binary_crossentropy"], )

history = model.fit(train_model_input, train[target].values, batch_size=32, epochs=10, verbose=2,
                    validation_split=0.2)
pred_ans = model.predict(test_model_input, 256)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

cuda
Train on 128 samples, validate on 32 samples, 4 steps per epoch
Epoch 1/10
0s - loss:  0.6845 - binary_crossentropy:  0.6845 - val_binary_crossentropy:  0.6712
Epoch 2/10
0s - loss:  0.6475 - binary_crossentropy:  0.6475 - val_binary_crossentropy:  0.6494
Epoch 3/10
0s - loss:  0.6144 - binary_crossentropy:  0.6144 - val_binary_crossentropy:  0.6313
Epoch 4/10
0s - loss:  0.5839 - binary_crossentropy:  0.5839 - val_binary_crossentropy:  0.6144
Epoch 5/10
0s - loss:  0.5505 - binary_crossentropy:  0.5505 - val_binary_crossentropy:  0.6004
Epoch 6/10
0s - loss:  0.5217 - binary_crossentropy:  0.5217 - val_binary_crossentropy:  0.5914
Epoch 7/10
0s - loss:  0.4895 - binary_crossentropy:  0.4895 - val_binary_crossentropy:  0.5886
Epoch 8/10
0s - loss:  0.4662 - binary_crossentropy:  0.4662 - val_binary_crossentropy:  0.5919
Epoch 9/10
0s - loss:  0.4358 - binary_crossentropy:  0.4358 - val_binary_crossentropy:  0.5958
Epoch 10/10
0s - loss:  0.4044 - binary_crossentropy:  0.4044 - val

# 5. Deep & Cross Network <a id="5"></a> <br>

In [ ]:
from deepctr_torch.models import DCN

model = DCN(linear_feature_columns, dnn_feature_columns, cross_num=2, cross_parameterization='vector', dnn_hidden_units=(256, 128), dnn_activation='relu', 
            task='binary', l2_reg_embedding=1e-5, l2_reg_dnn=0, device=device)

model.compile("adam", loss="binary_crossentropy", metrics=["binary_crossentropy"], )

history = model.fit(train_model_input, train[target].values, batch_size=32, epochs=10, verbose=2,
                    validation_split=0.2)
pred_ans = model.predict(test_model_input, 256)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

cuda
Train on 128 samples, validate on 32 samples, 4 steps per epoch
Epoch 1/10
0s - loss:  0.6799 - binary_crossentropy:  0.6799 - val_binary_crossentropy:  0.6632
Epoch 2/10
0s - loss:  0.6299 - binary_crossentropy:  0.6299 - val_binary_crossentropy:  0.6357
Epoch 3/10
0s - loss:  0.5865 - binary_crossentropy:  0.5865 - val_binary_crossentropy:  0.6137
Epoch 4/10
0s - loss:  0.5547 - binary_crossentropy:  0.5547 - val_binary_crossentropy:  0.5981
Epoch 5/10
0s - loss:  0.5237 - binary_crossentropy:  0.5237 - val_binary_crossentropy:  0.5930
Epoch 6/10
0s - loss:  0.4988 - binary_crossentropy:  0.4988 - val_binary_crossentropy:  0.5961
Epoch 7/10
0s - loss:  0.4730 - binary_crossentropy:  0.4730 - val_binary_crossentropy:  0.5989
Epoch 8/10
0s - loss:  0.4470 - binary_crossentropy:  0.4470 - val_binary_crossentropy:  0.5992
Epoch 9/10
0s - loss:  0.4174 - binary_crossentropy:  0.4174 - val_binary_crossentropy:  0.5958
Epoch 10/10
0s - loss:  0.3807 - binary_crossentropy:  0.3807 - val

# 6. Neural Factorization Machines, NFM <a id="6"></a> <br>

In [ ]:
from deepctr_torch.models import NFM

model = NFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 128), dnn_activation='relu', 
            task='binary', l2_reg_embedding=1e-5, l2_reg_dnn=0, device=device)

model.compile("adam", loss="binary_crossentropy", metrics=["binary_crossentropy"], )

history = model.fit(train_model_input, train[target].values, batch_size=32, epochs=10, verbose=2,
                    validation_split=0.2)
pred_ans = model.predict(test_model_input, 256)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

cuda
Train on 128 samples, validate on 32 samples, 4 steps per epoch
Epoch 1/10
0s - loss:  0.6692 - binary_crossentropy:  0.6692 - val_binary_crossentropy:  0.6460
Epoch 2/10
0s - loss:  0.6136 - binary_crossentropy:  0.6136 - val_binary_crossentropy:  0.6173
Epoch 3/10
0s - loss:  0.5715 - binary_crossentropy:  0.5715 - val_binary_crossentropy:  0.5985
Epoch 4/10
0s - loss:  0.5501 - binary_crossentropy:  0.5501 - val_binary_crossentropy:  0.5908
Epoch 5/10
0s - loss:  0.5264 - binary_crossentropy:  0.5264 - val_binary_crossentropy:  0.5913
Epoch 6/10
0s - loss:  0.5192 - binary_crossentropy:  0.5192 - val_binary_crossentropy:  0.5945
Epoch 7/10
0s - loss:  0.5097 - binary_crossentropy:  0.5097 - val_binary_crossentropy:  0.5961
Epoch 8/10
0s - loss:  0.4970 - binary_crossentropy:  0.4970 - val_binary_crossentropy:  0.5936
Epoch 9/10
0s - loss:  0.4833 - binary_crossentropy:  0.4833 - val_binary_crossentropy:  0.5883
Epoch 10/10
0s - loss:  0.4637 - binary_crossentropy:  0.4637 - val

# 7. DeepFM <a id="7"></a> <br>

In [ ]:
from deepctr_torch.models import DeepFM

model = DeepFM(linear_feature_columns, dnn_feature_columns, use_fm=True, dnn_hidden_units=(256, 128), dnn_activation='relu', 
            task='binary', l2_reg_embedding=1e-5, l2_reg_dnn=0, device=device)

model.compile("adam", loss="binary_crossentropy", metrics=["binary_crossentropy"], )

history = model.fit(train_model_input, train[target].values, batch_size=32, epochs=10, verbose=2,
                    validation_split=0.2)
pred_ans = model.predict(test_model_input, 256)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

cuda
Train on 128 samples, validate on 32 samples, 4 steps per epoch
Epoch 1/10
0s - loss:  0.6846 - binary_crossentropy:  0.6846 - val_binary_crossentropy:  0.6712
Epoch 2/10
0s - loss:  0.6477 - binary_crossentropy:  0.6477 - val_binary_crossentropy:  0.6496
Epoch 3/10
0s - loss:  0.6142 - binary_crossentropy:  0.6142 - val_binary_crossentropy:  0.6321
Epoch 4/10
0s - loss:  0.5838 - binary_crossentropy:  0.5838 - val_binary_crossentropy:  0.6160
Epoch 5/10
0s - loss:  0.5512 - binary_crossentropy:  0.5512 - val_binary_crossentropy:  0.6028
Epoch 6/10
0s - loss:  0.5230 - binary_crossentropy:  0.5230 - val_binary_crossentropy:  0.5935
Epoch 7/10
0s - loss:  0.4914 - binary_crossentropy:  0.4914 - val_binary_crossentropy:  0.5889
Epoch 8/10
0s - loss:  0.4661 - binary_crossentropy:  0.4661 - val_binary_crossentropy:  0.5884
Epoch 9/10
0s - loss:  0.4364 - binary_crossentropy:  0.4364 - val_binary_crossentropy:  0.5889
Epoch 10/10
0s - loss:  0.4052 - binary_crossentropy:  0.4052 - val